In [1]:
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from collections import namedtuple
import random
from Arm import Arm
from runner import graph_history, run_round

In [2]:

class BayesianArm(Arm):
    def __init__(self, name, expected_win_rate):
        super(BayesianArm, self).__init__(name, expected_win_rate)
        self.alpha = 1
        self.beta = 1
        
    
    def update_after_round(self, won):
        self.alpha += won
        self.beta += not won

        

In [3]:
def choose_arm(arms):
    winner_index = np.argmax([
        stats.beta.rvs(arm.alpha, arm.beta) for arm in arms
    ])
    return arms[winner_index]


In [4]:
min_value = 0.001

def graph_arms(probable_arms):
    x = np.linspace(0, 1, 1000)
    df = pd.DataFrame(index=x)
    for arm in probable_arms:
        df[arm.name] = stats.beta.pdf(x, arm.alpha, arm.beta)

    max_row = len(x)
    for index, row in df[::-1].iterrows():
        if any([r > min_value for r in row]):
            max_row = index
            break
        
    ax = sns.lineplot(data=df[0:max_row])
    for line in ax.lines:
        ax.fill_between(line.get_xydata()[:,0],line.get_xydata()[:,1], alpha=0.3)
    plt.autoscale(tight=True)
    plt.legend(title="Probability")
    plt.show()
    
    
    df = pd.DataFrame(index=x)
    for arm in probable_arms:
        df[arm.name] = stats.beta.cdf(x, arm.alpha, arm.beta)

        
    ax = sns.lineplot(data=df[0:max_row])
    for line in ax.lines:
        ax.fill_between(line.get_xydata()[:,0],line.get_xydata()[:,1], alpha=0.3)
    plt.autoscale(tight=True)
    plt.legend(title="Likelihood (Y) a random number is < X")
    plt.show()

In [5]:
def run(num_trials, probs):
    prob_list = [float(i.strip()) for i in probs.split(',')]
    
    arms = [BayesianArm(str(prob), prob) for prob in prob_list]
    arms_to_history = {arm: [] for arm in arms}
    
    for i in range(num_trials):
        arms_to_history = run_round(choose_arm, arms_to_history)

    graph_arms(arms)
    graph_history(arms_to_history)
    

In [6]:
widget = interact_manual(run, 
                     num_trials=widgets.IntSlider(min=1, max=1000, step=1, value=10, continuous_update=False), 
                     probs=widgets.Text(
                        placeholder='Comma-sep floats',
                        description='Probabilities:',
                        value='0.05,0.075,0.1',
                        continuous_update=False
                    ));
widget

interactive(children=(IntSlider(value=10, continuous_update=False, description='num_trials', max=1000, min=1),…

<function __main__.run(num_trials, probs)>